# Pytest Fixtures


@crlane
Tea Time 2016-09-29

## What is Pytest?

### Test runner

- discovery and collection

- execution

- reporting

In [7]:
!py.test -v . -k test_is_req_satisfied --collect-only

============================= test session starts ==============================
platform linux -- Python 3.5.2, pytest-3.0.2, py-1.4.31, pluggy-0.3.1 -- /opt/conda/bin/python
cachedir: .cache
rootdir: /home/jovyan/work, inifile: 
collected 12 items 
<Module 'test_file.py'>
  <Function 'test_is_req_satisfied[1.10.0-1.10.0-True]'>
  <Function 'test_is_req_satisfied[1.10.0-1.10.1-True]'>
  <Function 'test_is_req_satisfied[1.10.0-1.12.0-True]'>
  <Function 'test_is_req_satisfied[1.10.0-2.0.0-True]'>
  <Function 'test_is_req_satisfied[1.10.0-1.9.9-False]'>
  <Function 'test_is_req_satisfied[1.10.0-0.9.9-False]'>
  <Function 'test_is_req_satisfied[1.10.1-1.10.0-False]'>

============================== 5 tests deselected ==============================
========================= 5 deselected in 0.09 seconds =========================


- offers great flexibility

- unittest, nosetests, doctests

- extensible (hi pluggy!)

### Framework for authoring tests
  

- small tests, functional style

- more tests with less code to maintain
  

- bare asserts 

- better failure context

In [2]:
import pytest
@pytest.mark.parametrize(('minimum_ver', 'current', 'desired_bool'), [
    ("1.10.0", "1.10.0", True),
    ("1.10.0", "1.10.1", True),
    ("1.10.0", "1.12.0", True),
    ("1.10.0", "2.0.0", True),
    ("1.10.0", "1.9.9", False),
    ("1.10.0", "0.9.9", False),
    ("1.10.1", "1.10.0", False)
])
def test_is_req_satisfied(minimum_ver, current, desired_bool):
        assert desired_bool == is_req_satisfied(minimum_ver, current)

![failure](resources/failure_context.png)

### Framework for creating _fixtures_

- an item needed to create, model, or manage state during testing

- composable (build fixtures from fixtures)

- parametrizable (many cases, less code, easier to maintain)

- resolved at runtime (PATH is test file, conftest.py)

- can feel like (too much) magic, but once you grok it...

## Writing Your Own Fixtures

- Before you write, check the docs
  - `tmpdir` and `tmpdir_factory`
  - `monkeypatch` (replacement for `mock.patch`)
  - `capsys` and `capfd` (grab output or logging)

In [2]:
@pytest.fixture
def my_fixture(tmpdir):
    p = tmpdir.mkdir('foo').join('bar.txt')
    p.write('Fixtures improve tests')
    return p.strpath

In [9]:
def test_simple_fixture(my_fixture):
    with open(my_fixture) as f:
        assert simple_read(f) == 'Fixtures improve tests'

In [9]:
!py.test -v . -k test_simple_fixture

============================= test session starts ==============================
platform linux -- Python 3.5.2, pytest-3.0.2, py-1.4.31, pluggy-0.3.1 -- /opt/conda/bin/python
cachedir: .cache
rootdir: /home/jovyan/work, inifile: 
collected 8 items 

test_file.py::test_simple_fixture PASSED

============================== 7 tests deselected ==============================
==================== 1 passed, 7 deselected in 0.10 seconds ====================


## Parametrizing Fixtures

In [3]:
import json
import yaml
import pytest
import os

APPROVED_EXTS = [".txt", ".json", ".yaml"]


@pytest.fixture(params=APPROVED_EXTS + [".unknown"])
def param_file(request, tmpdir):
    file_type = request.param
    message = 'Fixtures improve tests'
    p = tmpdir.mkdir('foo').join('bar{}'.format(file_type))
    if file_type == '.txt':
        p.write(message)
    elif file_type == '.json':
        p.write(json.dumps({'message': message}))
    elif file_type == '.yaml':
        p.write(yaml.dump({'mEssage': message}))
    else:
        p.write('Unknown file type')
    return p.strpath

In [6]:
def test_param_fixture(param_file):
    if os.path.splitext(param_file)[-1] not in APPROVED_EXTS:
        with pytest.raises(TypeError):
            param_read(param_file)
    else:
        assert param_read(param_file) == 'Fixtures improve tests'

In [5]:
!pytest -v . -k test_param_fixture

============================= test session starts ==============================
platform linux -- Python 3.5.2, pytest-3.0.2, py-1.4.31, pluggy-0.3.1 -- /opt/conda/bin/python
cachedir: .cache
rootdir: /home/jovyan/work, inifile: 
collected 12 items 

test_file.py::test_param_fixture[.txt] PASSED
test_file.py::test_param_fixture[.json] PASSED
test_file.py::test_param_fixture[.yaml] PASSED
test_file.py::test_param_fixture[.unknown] PASSED

============================== 8 tests deselected ==============================
==================== 4 passed, 8 deselected in 0.12 seconds ====================


### Finalizer Callbacks

- reuse fixtures without side effects

In [12]:
@pytest.fixture(params=['api_v1', 'api_v2'])
def finalizer_fixture(request):
    
    def _cleanup_something():
        print("I'm in your fixture cleaning up")
        
    request.addfinalizer(_cleanup_something)    
    

In [15]:
!py.test -v . -k test_show_finalizer

============================= test session starts ==============================
platform linux -- Python 3.5.2, pytest-3.0.2, py-1.4.31, pluggy-0.3.1 -- /opt/conda/bin/python
cachedir: .cache
rootdir: /home/jovyan/work, inifile: 
collected 14 items 

test_file.py::test_show_finalizer[api_v1] PASSED
test_file.py::test_show_finalizer[api_v1] ERROR
test_file.py::test_show_finalizer[api_v2] PASSED
test_file.py::test_show_finalizer[api_v2] ERROR

==================================== ERRORS ====================================
_______________ ERROR at teardown of test_show_finalizer[api_v1] _______________

    def _cleanup_something():
>       raise ZeroDivisionError
E       ZeroDivisionError

test_file.py:86: ZeroDivisionError
_______________ ERROR at teardown of test_show_finalizer[api_v2] _______________

    def _cleanup_something():
>       raise ZeroDivisionError
E       ZeroDivisionError

test_file.py:86: ZeroDivisionError
============================= 12 tests deselected ==========

## Advanced Fixtures

- yield fixtures

- works like @contextlib.contextmanager without the decorator

- composing fixtures

- generate lots of tests

In [ ]:
@pytest.fixture(params=['North', 'East', 'West', 'South'])
def direction(request):
    return request.param

@pytest.fixture(params=['Carolina', 'Dakota'])
def state(request):
    return request.param

@pytest.yield_fixture
def composed_fixture(direction, state):
    directory = '/tmp/states/{} {}'.format(direction, state)
    try:
        os.mkdir(directory)
        yield directory
    finally:
        os.rmdir(directory)

def test_directional_state(composed_fixture):
    assert os.path.exists(composed_fixture)
    assert len(os.listdir('/tmp/states')) == 1

In [21]:
!pytest -v . -k test_directional_state

============================= test session starts ==============================
platform linux -- Python 3.5.2, pytest-3.0.2, py-1.4.31, pluggy-0.3.1 -- /opt/conda/bin/python
cachedir: .cache
rootdir: /home/jovyan/work, inifile: 
collected 22 items 

test_file.py::test_directional_state[North-Carolina] PASSED
test_file.py::test_directional_state[North-Dakota] PASSED
test_file.py::test_directional_state[East-Carolina] PASSED
test_file.py::test_directional_state[East-Dakota] PASSED
test_file.py::test_directional_state[West-Carolina] PASSED
test_file.py::test_directional_state[West-Dakota] PASSED
test_file.py::test_directional_state[South-Carolina] PASSED
test_file.py::test_directional_state[South-Dakota] PASSED

============================= 14 tests deselected ==============================
=================== 8 passed, 14 deselected in 0.13 seconds ====================
